[과제1]
[수행할 작업]
1. [수정할 코드]들을 수정해서 제출해 주세요. 
     - sqlite3을 이용해서 데이터를 추출, 적재, 화면출력하는 함수를 갖고 있는 database.py 코드를 생성
     - database.py를 호출해서 [수정할 코드]들을 수정 --> 코드 간소화 
     - create_tables.py 에서 predict 테이블을 생성할때 predict 컬럼만 생성하지 말고 X컬럼들도 같이 적재할수 있도록 준비. 
     - 3.batch_run.py, run.py를 수정해서 predict테이블에 추론할 X컬럼들과 추론결과(predict)가 적재 되도록 수정.
2. run.py에서 100건의 online추론이 진행되도록 수정해 주세요. (inference.py도 수정해야 효율화가 됨)
    - Test 테이블에서 랜덤하게 데이터 1개 추출해서 추론한 후 predict 테이블에 적재 --> 100번 반복 --> 100개의 추론결과가 predict에 적재되도록
    - 추론을 실행한 후 predict테이블에 X값들과 추론결과(predict)가 같이 적재되도록 수정
3. 코드들이 완성되면, 다음의 순서로 실행했을 때 에러 없이 실행되어야 합니다. 
    - python3 1.create_tables.py
    - python3 2.train.py
    - python3 3.batch_run.py
    - python3 run.py 
4. 위 3을 진행한 후 steel.db 파일의 predict 테이블을 출력하면 
    X컬럼들과 predict컬럼이 출력되고, 행의 개수는 583+100=683개가 되어야 합니다. 
[수정할 코드]
①. "1.create_tables.py"
②. "2.train.py"
③. "3.batch_run.py"
④. "run.py" 
⑤ "inference.py"
[추가로 생성할 코드]
① "database.py" 
과제제출이 안되거나 질문 있으시면 메일로 연락부탁드립니다. 
파일제출형태 : 학번_과제1.zip의 형태로 파일 첨부하여 제출해주세요
jkim9747@gmail.com

In [ ]:
#%run 1.create_tables_my.py
#%run 2.train_my.py
#%run 3.batch_run_my.py
#%run run_my.py

In [ ]:
# %load 1.create_tables_my.py
import database
from database import change_dir, data2sql, create_table, print_table
import os

change_dir()
db_name = 'steel.db'
train_path = "data/train.csv"
test_path = "data/test.csv"

# create train/test table into DB
data2sql(db_path = db_name , data_path = train_path, table_name = "train")
data2sql(db_path = db_name, data_path = test_path, table_name = "test")

# create predict table
col_name ={"predict" : "TEXT" }
create_table(db_path = db_name, table_name = "predict", col_dict = col_name)

# print table
print_table(db_path = db_name, table_name='train')
print_table(db_path = db_name, table_name='test')
print_table(db_path = db_name, table_name='predict')

In [ ]:
# %load 2.train_my.py
import sqlite3
import pandas as pd
import numpy as np
import pickle as pkl
import database as db
from sklearn.model_selection import train_test_split

db.change_dir()
# Load data from sql
dat = db.sql2df(db_path="steel.db", table_name = "train")

## train, test split
from sklearn.model_selection import train_test_split
train, test = train_test_split(dat, test_size=0.3)
train.head()

## x variables preprocessing 
x_cols = ['V'+str(i) for i in range(1,28)]
from sklearn.preprocessing import StandardScaler
trans = StandardScaler()
trans.fit(train[x_cols])
train_x = trans.transform(train[x_cols])

## y variables preprocessing
train['V34'] = train['Class']-1
train_y = [str(np.where(r==1)[0][0]) for r in train[['V'+str(i) for i in range(28,35)]].to_numpy()]

## classification modeling 
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(train_x, train_y)

## save model and transformation 
print("creating model file...")
pkl.dump(model, open("model.pkl","wb"))
print("creating transformation file...")
pkl.dump(trans, open("transform.pkl","wb"))

## make prediction for testset
test_x = trans.transform(test[x_cols])
pred = model.predict(test_x)

## validation for testset
test['V34'] = test['Class']-1
test_y = [str(np.where(r==1)[0][0]) for r in test[['V'+str(i) for i in range(28,35)]].to_numpy()]
print(np.mean(pred==test_y))
print(pd.crosstab(test_y, pred))


In [ ]:
# %load 3.batch_run_my.py
import sqlite3
import pandas as pd
import numpy as np
import inference as inf
import database as db

db.change_dir()

## load test dataset
dat = db.sql2df(db_path="steel.db", table_name = 'test')

## make prediction for test set
print("\n\n make prediction...")
pred = inf.inference(dat)
dat["pred"] = pred


## insert prediction into predict table 
db.df2sql(df_name = dat, db_path = "steel.db", table_name = "predict") # Insert 하는 방식으로 수정해야 함

## print predict table
db.print_table(db_path="steel.db",table_name = "predict", nrow = 10)


In [ ]:
# %load run_my.py
import sqlite3
import pandas as pd
import numpy as np
import database as db

db.change_dir()


db_name = "steel.db"
conn = sqlite3.connect(db_name)
c = conn.cursor()
c.execute(""" SELECT name FROM sqlite_master WHERE type='table' """)
print(c.fetchall())

c.execute(""" SELECT count(*) FROM train """)
print(c.fetchall())
c.execute(""" SELECT count(*) FROM test """)
print(c.fetchall())
c.execute(""" SELECT count(*) FROM predict """)
print(c.fetchall())

# 100개의 온라인 추론
# 1. test 테이블에서 랜덤하게 데이터 100개 추출
#idx = 1
#c.execute(""" SELECT * FROM test ORDER BY RANDOM() LIMIT 2""")

import call_func as cf
import inference as infer
ind = np.random.randint(0,583, size=100) # 100개의 랜덤한 행 인덱스 출력
for i in ind :
    dat = cf.call_x_value(str(i)) # test table에서 i번째 행의 데이터 불러오기
    pred = infer.inference(dat)[0]
    dat["pred"] = pred
    dat.to_sql("predict", conn, if_exists= "append", index = False)

print("After Appending 100 online inference")
c.execute(""" SELECT count(*) FROM predict """)
print(c.fetchall())
db.print_table("steel.db", "predict", nrow = 700)
#print_table(db_path = db_name, table_name='predict')